# Automated pipeline

### Importing data
We already have a pre-elaboration od the data thanks to Marta, i will work with this new dataset

In [1]:
import pandas as pd

diamonds = pd.read_csv("../data/diamonds_preprocessed.csv")

## Creating the models dictionary

My goal is to create a easy and flexible solution ready for future changes, so i will simplify everything by using a dictionary of models

Marta has her notebook where she trains and analyzes the models, so i will use her models to create the dictionary, i will use Pickle for saving the models and importing them so that the mangement of the two is easier

In [2]:
# import every model using pickle into a dictionary, by importing every pickle in the models folder

import pickle,os

model_path = "../history/current_models"
models = {}

# Load the models from the models folder

for file in os.listdir(model_path):
    if file.endswith(".pkl"):
        model_name = file.split(".")[0]
        model = pickle.load(open(f"{model_path}/{file}", "rb"))
        models[model_name] = model

for model_name, model in models.items():
    print(f"Model: {model_name}")
    print(f"Model: {model}")
    

Model: linear_regression_diamonds
Model: {'model': LinearRegression(), 'model_name': 'linear_regression_diamonds', 'preferred_preprocessing': 0}
Model: xgboost_diamonds
Model: {'model': XGBRegressor(alpha=0.15199357133143715, base_score=None, booster=None,
             callbacks=None, colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.7, device=None, early_stopping_rounds=None,
             enable_categorical=True, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, lambda=0.0011030416763866523,
             learning_rate=0.03238176135427902, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=5, max_leaves=None,
             min_child_weight=7, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=274, n_jobs=None, ...), 'model_name': 'xgboost_diamonds', 'preferred_p

## Preprocessing the new data

We need to preprocess the new data in different ways, based on the model we are training

I changed a bit the notebook made by Marta by adding the creation of dictionaries which contain the models and their preferences about data, for example either if it likes non numerical values or not

In [3]:
from copy import deepcopy
def preprocess_input_for_tree_models(input_data):
    diamonds_processed = deepcopy(input_data)
    # remove every line with a missing value
    diamonds_processed = diamonds_processed[(diamonds_processed.x * diamonds_processed.y * diamonds_processed.z != 0) & (diamonds_processed.price > 0)]
    diamonds_processed['cut'] = pd.Categorical(diamonds_processed['cut'], categories=['Fair', 'Good', 'Very Good', 'Ideal', 'Premium'], ordered=True)
    diamonds_processed['color'] = pd.Categorical(diamonds_processed['color'], categories=['D', 'E', 'F', 'G', 'H', 'I', 'J'], ordered=True)
    diamonds_processed['clarity'] = pd.Categorical(diamonds_processed['clarity'], categories=['IF', 'VVS1', 'VVS2', 'VS1', 'VS2', 'SI1', 'SI2', 'I1'], ordered=True)
    return diamonds_processed

def preprocess_input_for_linear_models(input_data):
    diamonds_processed = deepcopy(input_data)
    diamonds_processed = diamonds_processed[(diamonds_processed.x * diamonds_processed.y * diamonds_processed.z != 0) & (diamonds_processed.price > 0)]
    diamonds_processed = diamonds_processed.dropna()
    diamonds_processed.drop(columns=['depth', 'table', 'y', 'z'])
    diamonds_processed = pd.get_dummies(diamonds_processed, columns=['cut', 'color', 'clarity'])
    return diamonds_processed
        

## Fit the current models with the new data 

In [4]:
from sklearn.model_selection import train_test_split
import numpy as np
import utils

validation_df = pd.read_csv("../data/diamonds_1.csv")
validation_0 = preprocess_input_for_linear_models(validation_df)
validation_1 = preprocess_input_for_tree_models(validation_df)

x_val_0 = validation_0.drop(columns=['price'])
y_val_0 = validation_0['price']
x_val_1 = validation_1.drop(columns=['price'])
y_val_1 = validation_1['price']

def fit_test_models(new_data):
    print("Fitting models")
    x_train_0 = new_data[0].drop(columns=['price'])
    y_train_0 = new_data[0]['price']
    x_train_1 = new_data[1].drop(columns=['price'])
    y_train_1 = new_data[1]['price']
    for model_name,model in models.items():
        if(model["preferred_preprocessing"] == 0):
            model["model"].fit(x_train_0, np.log(y_train_0))
            utils.save_model(model, model_name)
        elif(model["preferred_preprocessing"] == 1):
            model["model"].fit(x_train_1, y_train_1)
            utils.save_model(model, model_name,1)
        else:
            print(model["preferred_preprocessing"])
            raise ValueError("Invalid preprocessing method")
    test_models()

def test_models():
    results = {}
    for model_name,model in models.items():
        if(model["preferred_preprocessing"] == 0):
            pred = np.exp(model["model"].predict(x_val_0))
            utils.save_scores(y_val_0, pred, model_name)
        elif(model["preferred_preprocessing"] == 1):
            pred = model["model"].predict(x_val_1)
            utils.save_scores(y_val_1, pred, model_name)
        else:
            print(model["preferred_preprocessing"])
            raise ValueError("Invalid preprocessing method")
    return results

## Scanning folder for new data

In [5]:
!pip install watchdog

In [6]:
# new_data_preprocessed = []
# file_path = "../data/new_data/diamonds_2.csv"

# new_data = pd.read_csv(file_path)
# new_data_preprocessed_tree = preprocess_input_for_tree_models(new_data)
# new_data_preprocessed_linear = preprocess_input_for_linear_models(new_data)
# new_data_preprocessed.append(new_data_preprocessed_linear)
# new_data_preprocessed.append(new_data_preprocessed_tree)


In [7]:
# new_data_preprocessed[1]

In [8]:
# fit_test_models(new_data_preprocessed)

In [9]:
import time
from watchdog.observers import Observer
from watchdog.events import FileSystemEventHandler

class FileEventHandler(FileSystemEventHandler):
    def on_created(self, event):
        if event.is_directory:
            return
        if event.src_path.endswith('.csv'):
            print(f"New file detected: {event.src_path}")
            process_new_file(event.src_path)

def process_new_file(file_path):
    new_data_preprocessed = []
    # Add your data processing and model training code here
    print(f"Processing file: {file_path}")
    new_data = pd.read_csv(file_path)
    new_data_preprocessed_tree = preprocess_input_for_tree_models(new_data)
    new_data_preprocessed_linear = preprocess_input_for_linear_models(new_data)
    new_data_preprocessed.append(new_data_preprocessed_linear)
    new_data_preprocessed.append(new_data_preprocessed_tree)
    
    #print('new data 0', new_data_preprocessed[0].head())
    fit_test_models(new_data_preprocessed)


if __name__ == "__main__":
    path = "../data/new_data"
    event_handler = FileEventHandler()
    observer = Observer()
    observer.schedule(event_handler, path, recursive=False)
    observer.start()
    
    try:
        while True:
            time.sleep(1)
    except KeyboardInterrupt:
        observer.stop()
    observer.join()



New file detected: /Users/leonardovaia/Documents/Job_interview/xtream-ai-assignment-developer/data/new_data/diamonds_2.csv
Processing file: /Users/leonardovaia/Documents/Job_interview/xtream-ai-assignment-developer/data/new_data/diamonds_2.csv
Fitting models
linear_regression_diamonds
{'r2_score': 0.9480622269624593, 'mae': 409.8306245856353}
xgboost_diamonds
{'r2_score': 0.9575192706274285, 'mae': 382.9641550730418}
New file detected: /Users/leonardovaia/Documents/Job_interview/xtream-ai-assignment-developer/data/new_data/diamonds_3.csv
Processing file: /Users/leonardovaia/Documents/Job_interview/xtream-ai-assignment-developer/data/new_data/diamonds_3.csv
Fitting models
linear_regression_diamonds
{'r2_score': 0.9383460275734579, 'mae': 425.2228152355467}
xgboost_diamonds
{'r2_score': 0.9532793502986119, 'mae': 406.11942951862994}
New file detected: /Users/leonardovaia/Documents/Job_interview/xtream-ai-assignment-developer/data/new_data/diamonds_4.csv
Processing file: /Users/leonardova

In [ ]:
process_new_file("../data/diamonds.csv")

Processing file: ../data/diamonds.csv
Fitting models


AttributeError: 'dict' object has no attribute 'fit'